Train gradient boosting models on the data.

# 1. Load data and packages

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from handyspark import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import RandomForestClassifier

Load the modeling helper functions

In [5]:
%run Users/tw2567@columbia.edu/Udacity/churn_prediction/util

In [6]:
raw_data = spark.read.parquet('s3://tianyi-wang-data-science-projects/churn-prediction-2020/features1')\
           .withColumnRenamed('last_status_cancelled','label')\
           .drop('userId')
features_col = [i for i in raw_data.columns if i != 'label']
assembler = VectorAssembler(inputCols=features_col,outputCol="features")
data = assembler.transform(raw_data).select('features','label')

In [7]:
feature_names = assembler.getInputCols()

In [8]:
display(data.groupby('label').count())

label,count
1,5005
0,17272


# 2 Check whether it's better to upsample the positive group

### 2.1 Triple the size of positive samples

In [11]:
auc = tune_sampling_parameters(data, RandomForestClassifier(), 3, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 10510.5
Majority -- training size went from 12090.4 to 12090.4
fold 0
10453
11977
fold 1
10612
11954
fold 2
10523
11978
Out[7]: [0.7472955492796227, 0.7467754735225729, 0.7489651167530008]

### 2.2 Double the size of positive samples

In [13]:
auc = tune_sampling_parameters(data, RandomForestClassifier(), 2, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 7007.0
Majority -- training size went from 12090.4 to 12090.4
fold 0
7084
11955
fold 1
6989
11984
fold 2
6940
11969
Out[9]: [0.7341494247115419, 0.7422799066273792, 0.7473017954885213]

### 2.3 Postive group size * 1.5

In [15]:
auc = tune_sampling_parameters(data, RandomForestClassifier(), 1.5, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 5255.25
Majority -- training size went from 12090.4 to 12090.4
fold 0
5212
11964
fold 1
5084
11954
fold 2
5144
11965
Out[10]: [0.7362106105549321, 0.7330536251545308, 0.7361719345341787]

We decide to triple the size of the positive group.

### 2.4 Don't upsample

In [18]:
auc = tune_sampling_parameters(data, RandomForestClassifier(), 1, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 3503.5
Majority -- training size went from 12090.4 to 12090.4
fold 0
3532
11964
fold 1
3581
11971
fold 2
3570
11977
Out[11]: [0.7302525474752888, 0.7348536689915601, 0.7272939265271696]

# 3. Tune model parameters

In [20]:
rf = RandomForestClassifier()
paramGrid = ParamGridBuilder()\
            .addGrid(rf.maxDepth, [5, 8])\
            .addGrid(rf.subsamplingRate, [0.8, 1])\
            .build()
cv_model = tune_model_parameter(data, 3, 1, rf, paramGrid)

In [21]:
rf = RandomForestClassifier()
paramGrid = ParamGridBuilder()\
            .addGrid(rf.maxDepth, [5, 8])\
            .addGrid(rf.subsamplingRate, [0.8, 1])\
            .build()
cv_model = tune_model_parameter(data, 3, 1, rf, paramGrid)

In [22]:
rf = RandomForestClassifier()
paramGrid = ParamGridBuilder()\
            .addGrid(rf.maxDepth, [5, 8])\
            .addGrid(rf.subsamplingRate, [0.8, 1])\
            .build()
cv_model = tune_model_parameter(data, 3, 1, rf, paramGrid)

In [23]:
cv_model.bestModel.extractParamMap()

Out[9]: {Param(parent='RandomForestClassifier_85fc1b1c5281', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'auto',
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='featuresCol', doc='features column name'): 'features',
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini',
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='labelCol', doc='label column name'): 'label',
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be at least 2 and at least number of categories for any categorical feature.'): 32,
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='maxDepth', doc='Maximum depth of the tree. (Nonnegative) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 8,
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation.'): 256,
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Must be at least 1.'): 1,
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='numTrees', doc='Number of trees to train (at least 1)'): 20,
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities'): 'probability',
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name'): 'rawPrediction',
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='seed', doc='random seed'): -5387697053847413545,
 Param(parent='RandomForestClassifier_85fc1b1c5281', name='subsamplingRate', doc='Fraction of the training data used for learning each decision tree, in range (0, 1].'): 1.0}